# Analiza vpliva centralizacije na občine v Sloveniji

Cilj naloge je ugotoviti katere občine bi pridobile največ z decentralizacijo delovnih mest v Sloveniji. V nalogi bom analiziral migracije med Slovenskimi občinami in se naslajal na podatke o zaslužkih ljudi v danih občinah, njihovih migracijah ter cen nepremičnin. S temi podatki bom ugotavljal v katerih občinah bi bilo mogoče najbolj izboljšat zaslužek in standard življenja. 

## Uvoz podatkov

Na začetku naloge bom iz zbranih podatkov izračunal kazalnike, ki vplivajo na posamezno občino. S tem namenom naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [22]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 20

%matplotlib inline

# Podatki
import os.path

PODATKI = "./podatki"

nepremicnine = pd.read_csv(os.path.join(PODATKI, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(PODATKI, "podjetja.csv"), index_col="id")
razdalje = pd.read_csv(os.path.join(PODATKI, "razdalje.csv"), index_col="zacetek")
migracije_v_obcino = pd.read_csv(os.path.join(PODATKI, "migracije_v_obcino.csv"), index_col=[0])
migracije_iz_obcine = pd.read_csv(os.path.join(PODATKI, "migracije_iz_obcine.csv"), index_col=[0])
obcine = pd.read_csv(os.path.join(PODATKI, "obcine.csv"), index_col="ime")

Poglejmo najprej s kakšnimi podatki bomo delali. Naložili smo podatke o nepremičninah, ki vsebujejo podatke o vrsti nepremičnine, ceni, vrsti posredovanja, številu sob in podobno. Prav tako sem zbral podatke o razdaljah med posameznimi občinami in migracijah, ki nam bodo pomagale ocenit koliko povprečno posameznik izgubi časa in denarja zaradi potrebnih poti.

Poleg teh podatkov, imamo še informacije o večjih Slovenskih podjetjih, kar nam bo pomagalo ocenit pri katera podjetja delajo največjo izgubo s trenutno ureditvijo.

Vse podatke pa bomo združevali po občinah, zato imamo tudi statistične podatke o posamezni občini.

In [6]:
nepremicnine.info()
podjetja.info()
migracije_iz_obcine.info()
migracije_v_obcino.info()
razdalje.info()
obcine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24934 entries, o6358608 to o6353469
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   posredovanje   24934 non-null  object 
 1   naslov         24934 non-null  object 
 2   url            24934 non-null  object 
 3   kratek_opis    24934 non-null  object 
 4   vrsta          24933 non-null  object 
 5   regija         24933 non-null  object 
 6   upravna_enota  24933 non-null  object 
 7   obcina         24757 non-null  object 
 8   velikost       24925 non-null  float64
 9   cena           24930 non-null  float64
 10  leto           20022 non-null  float64
 11  sobe           5450 non-null   float64
 12  agencija       24934 non-null  object 
dtypes: float64(4), object(9)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9925 entries, 0 to 9924
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------          

## Obdelava podatkov o nepremičninah

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

Cene nepremičnin nam bodo pomagale ocenit, če je razlika v ceni nepremičnine, glede na to kam ljudje iz te občine migrirajo, smiselna glede na čas in stroške poti. V idealni situaciji bi živel tam, kjer je cena nepremičnin majhna, pot kratka, zaslužek pa velik.

In [7]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Zajem
nepremicnine_po_obcinah = nepremicnine[
    nepremicnine.vrsta.isin(["Hiša", "Stanovanje"]) & 
    (nepremicnine.posredovanje == "prodaja")
].groupby("obcina")
    
# Obdelaj in priključi podatke k občinam.
obcine["cena_na_m2"] = nepremicnine_po_obcinah["cena_na_m2"].mean()
obcine["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

## Obdelava podatkov o migracijah

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah. Zanima nas, kakšen delež migrantov iz posamezne regije gre povprečno v dano drugo regijo. Povedano drugače, zanima nas kakšna je verjetnost, da oseba, ki živi v Ljubljani in migrira, migrira v Maribor, Koper...

To bomo izračunali na sledeč način:

1. Najprej bomo izračunali absolutno število ljudi, ki vsak dan migrira v Ljubljano. Ta podatek dobimo kot razliko delavno aktivnih prebivalcev, ki delajo v Ljubljani (po delovnem mestu) in delavno aktivnih prebivalcev, ki živijo v Ljubljani (po prebivališču), pomnožen z deležem lokalnih delavcev.

1. Glede na izbrano občino bomo izračunali kakšna je verjetnost, da gre ta oseba v druge občine. To bomo določili kot relativno število migracije v posamezno občino glede na vse ostale občine razen te, iz katere oseba prihaja. Ker ljudje raje delajo bližje doma, bom vrednosti obtežil glede na oddaljenost od dane občine.


In [15]:
# Izračunamo koliko časa in kilometrov v povprečju posameznik iz dane občine porabi.
def pot(obcina):
    migracije = migracije_iz_obcine[obcina]
    poti = razdalje[razdalje.index == obcina].set_index("konec")

    df = pd.DataFrame({ 
        "razdalja": poti.razdalja, 
        "cas": poti.cas, 
        "migracije": migracije
    }, index=migracije.index)

    df["rel_razdalja"] = df.razdalja * df.migracije
    df["rel_cas"] = df.cas * df.migracije
    
    return pd.Series(
        [
            df["rel_razdalja"].sum(), 
            df["rel_cas"].sum()
        ], 
        index=["razdalja", "cas"]
    )


# Izračunaj na vseh občinah.
vektor_obcin = pd.Series(obcine.index, index=obcine.index)
poti = pd.DataFrame(vektor_obcin.apply(pot), columns=["razdalja", "cas"])

# Dodaj k občinam
obcine["razdalja_migracije"] = poti.razdalja
obcine["cas_migracije"] = poti.cas


# poti.sort_values("razdalja").tail(20)
# poti["Grosuplje"], poti["Maribor"]
# migracije_iz_obcine["Grosuplje"].sort_values(ascending=False)

Na podlagi podatkov o migracijah lahko ocenimo koliko denarja posamezna občina nameni migracijam. To bom naredil tako, da bom na podlagi povprečne dolžine poti do delovnega mesta, izračunal koliko bencina porabijo ljudje za pot ter iz cene naftnih derivatov izračunal oceno potnih stroškov. Na podlagi povprečne neto plače v občini bom izračunal urno postavko delavca ter upošteval še povprečni čas vožnje na delovno mesto.

Povprečno porabo avtomobila sem pobral iz [te](https://www.iea.org/reports/fuel-consumption-of-cars-and-vans) strani in znaša malo več kot 7L/100km, kar bom zaokrožil na 7L/100km. Za ceno 1L goriva bom vzel kar 1€, saj je toliko znašala cena goriva v zadnjem letu.

Izračunal bom še znesek nadomestila, ki ga migrantje prejmejo od svojih podjetji kot obračun prevoza na delo. Povrnjen znesek iz [vira](https://mladipodjetnik.si/novice-in-dogodki/novice/obracun-prevoz-na-delo-ter-kilometrine-2) znaša 0.18€/km.

In [20]:
# Stroški prevoza

poraba_goriva = 7 / 100
cena_goriva = 1.0 / 1

delovni_dnevi_v_mescu = 4 * 5
st_potovanj = delovni_dnevi_v_mescu * 2

# Izračuni vrednosti

obcine["stroski_goriva"] = (obcine.razdalja_migracije / 1000) * poraba_goriva * cena_goriva * st_potovanj

# Čas potovanja
delovne_ure_v_mescu = delovni_dnevi_v_mescu * 8
obcine["povprecna_urna_postavka"] = obcine.placa_neto / delovne_ure_v_mescu

obcine["stroski_prevoza"] = (obcine.cas_migracije / 3600) * st_potovanj * obcine.povprecna_urna_postavka

# Povračilo potnih stroškov na prebivalca.
obcine["povracilo_stroskov_migracije"] = 0.18/1 * (obcine.razdalja_migracije / 1000) * st_potovanj

# Združimo vse stroške.
obcine["stroski_migracije_na_delovca"] = obcine.stroski_prevoza + obcine.stroski_goriva
obcine["razlika_v_stroskih_na_delovca"] = obcine.stroski_migracije_na_delovca - obcine.povracilo_stroskov_migracije



# Prikažemo rezultate
stroski_migracije = [
    "razdalja_migracije", 
    "stroski_goriva", 
    "cas_migracije", 
    "povprecna_urna_postavka", 
    "stroski_prevoza", 
    "stroski_migracije_na_delovca",
    "razlika_v_stroskih_na_delovca"
]

obcine.sort_values("stroski_migracije_na_delovca", ascending=False)[stroski_migracije]

,razdalja_migracije,stroski_goriva,cas_migracije,povprecna_urna_postavka,stroski_prevoza,stroski_migracije_na_delovca,razlika_v_stroskih_na_delovca
ime,,,,,,,
Bovec,138302.295805,387.246428,7117.431609,6.335562,501.032587,888.279015,-107.497515
Kobarid,113420.348262,317.576975,6767.519290,6.158250,463.067507,780.644483,-35.982025
Tolmin,100700.399118,281.961118,6083.934004,6.590875,445.538317,727.499435,2.456561
Kanal,114751.921885,321.305381,5452.272403,6.639375,402.218679,723.524060,-102.689778
Brda,114575.659990,320.811848,5349.558042,6.586000,391.468770,712.280618,-112.664134
...,...,...,...,...,...,...,...
Domžale,35626.281424,99.753588,2008.893138,6.525375,145.653123,245.406711,-11.102515
Škofljica,38026.250608,106.473502,1959.039684,6.170687,134.318019,240.791520,-32.997484
Trzin,34302.250559,96.046302,1836.001987,7.021250,143.233655,239.279957,-7.696247


## Obdelava podatkov o podjetjih

Ocenili bi radi katera podjetja bodo igrala ključno vlogo v posamezni občini pri doesganju decentralizacije. Predpostavil bom, da imajo podjejta vsa delovna mesta v občini registracije in poizkušal ocenit kakšen odstotek ljudi, ki dela v danem podjetju, prihaja iz posamezne občine.

In [23]:
podjetja

,ime,maticna_stevilka,obcina,zaposleni,dobicek
id,,,,,
1519140000,ADECCO H.R. d.o.o.,1519140000,Ljubljana,Nad 1000 zaposlenih,604403.0
5479541000,"ADRIA MOBIL, d.o.o. Novo mesto",5479541000,Novo mesto,Nad 1000 zaposlenih,35762836.0
1387570000,Akrapovič d.d.,1387570000,Ivančna Gorica,Nad 1000 zaposlenih,16674429.0
5518121000,BOXMARK LEATHER d.o.o.,5518121000,Kidričevo,Nad 1000 zaposlenih,-4750308.0
5684943000,BSH HIŠNI APARATI d.o.o. Nazarje,5684943000,Nazarje,Nad 1000 zaposlenih,13133119.0
...,...,...,...,...,...
5834945000,ŽNIDER'S d.o.o.,5834945000,Maribor,10 do 19 zaposlenih,42135.0
3879534000,ŽUPA d.o.o.,3879534000,Grosuplje,10 do 19 zaposlenih,797.0
2195437000,ŽUPNIJSKI ZAVOD SV. MARTINA KRANJ - STRAŽIŠČE,2195437000,Kranj,10 do 19 zaposlenih,NaN


## Združenje podatkov o nepremičninah in stroških migracije.


Naš cilj je ugotoviti katere občine imajo največji potencial za izboljšanje življenjskega standarda ljudi z decentralizacijo.
